# Semantische Ähnlichkeit von Texten

Natürliche Sprachen bieten uns mannigfaltige Möglichkeiten, dieselben Inhalte auf unterschiedliche Art und Weise auszudrücken. Für die Interpretation natürlichsprachlicher Texte reicht es daher nicht, auf den Ebenen von Morphologie und Syntax zu bleiben und die Form von Wörtern sowie deren Anordnung im Satz zu betrachten, sondern wir müssen als zusätzliche Ebene die der Semantik betrachten.

Dabei beginnen wir zunächst mit der Betrachtung der Wortebene und sammeln Erfahrung mit word2vec.

Anschließend wollen wir versuchen, von Wort- zu Satzbedeutungen zu kommen: Wie gut können wir die Ähnlichkeit zweier Sätze unter Rückgriff auf die semantischen Embeddings der in ihnen enthaltenen Wörter bestimmen?

Als Orientierung dient uns dabei das [STS-Benchmark-Datenset](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark), das im Rahmen von [SemEval](https://en.wikipedia.org/wiki/SemEval) entstanden ist.
Das Datenset enthält Trainings-, Test- und Evaluationsdaten zu folgender Aufgabenstellung: 
Gegeben zwei Sätze S1 und S2, berechne Ähnlichkeitswert zwischen 0 (völlig unterschiedlicher Inhalt) und 5 (bedeutungsgleich).

## Aufgabe 1: Word2Vec
> You shall know a word by the company it keeps.
>
> -- <cite>J. R. Firth</cite>

Im ersten NLP-Labor haben wir mit dem Bag-of-Words-Modell eine recht einfache Methode festgestellt, um Wörter bzw. ganze Sätze in einen Vektorraum abzubilden: Jedem Wort wird ein eindeutiger Index zugewiesen. Es lässt sich dann als one-hot-enkodierter Vektor darstellen, der an eben jenem Index eine Eins und ansonsten nur Nullen enthält und dessen Länge der Anzahl der Wörter im Vokabular entspricht. Die Enkodierung ganzer Sätze erhält man durch Aufsummieren dieser one-hot-enkodierten Vektoren.
Bei einem solchen Modell geht auf Satzebene die Information über die Reihenfolge der Wörter verloren und ganz allgemein wird offensichtlich die Semantik der Wörter nicht berücksichtigt.

Gehen wir zum Beispiel von folgendem Vokabular aus: ```{"heiß", "warm", "Gurke"}``` und enkodieren die einzelnen Wörter wie folgt: ```[1, 0, 0], [0, 1, 0], [0, 0, 1]```, dann sind sich "heiß" und "warm" genauso ähnlich oder unähnlich wie "heiß" und "Gurke".

Semantische Embeddings schaffen an dieser Stelle Abhilfe. Ein bekannter Vertreter dieser Ansätze ist das Word2Vec-Modell, das 2013 von [Tomas Mikolov et al.](https://arxiv.org/abs/1301.3781) bei Google entwickelt wurde und seither rasch an Verbreitung gewonnen hat.

Wir werden mit vortrainierten Google-News-Embeddings arbeiten, die [hier](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing) heruntergeladen werden können. Die Vektoren haben die Länge 300.

### Aufgabe 1.1: Vorbereitung
Ladet die Embeddings über den oben angegebenen Link herunter und verwendet gensim, um sie zu laden.

**Hinweis**: Kann einen Moment dauern, bis das Dictionary, das von Wort auf Embedding abbildet, erzeugt ist. Im Zweifel ein ```limit``` angeben und nur die ersten 1,5 Mio. Embeddings laden.

In [4]:
import gensim

embeddings = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
def check_embeddings(embeddings):
    error_message = "Oh, da ist wohl etwas schiefgelaufen! Der folgende Test schlägt fehl: {}."
    most_similar = embeddings.most_similar(positive=['man', 'queen'], negative=['woman'])
    if len(most_similar) < 1 or most_similar[0][0] != 'king':
        return error_message.format("'Most similar'")

    doesnt_match = embeddings.doesnt_match(['spring', 'rain', 'autumn', 'summer'])
    if doesnt_match != 'rain':
        return error_message.format("'Doesn't match'")
    
    most_similar_to_given = embeddings.most_similar_to_given('school', ['cat', 'sound', 'university', 'whine'])
    if most_similar_to_given != 'university':
        return error_message.format("'Most similar to given'")
    
    return "Sieht gut aus."

check_embeddings(embeddings)

'Sieht gut aus.'

### Aufgabe 1.2: Spaß mit Semantik
Im Folgenden wollen wir uns ein bisschen mit dem Mehrwert beschäftigen, den semantische Embeddings bieten.

Mit ihnen lassen sich zum Beispiel folgende Fragen beantworten:


In [34]:
# Welche Stadt ist das New York Deutschlands? (Hinweis: 'New_York' ist als Token in den Embeddings enthalten)
print('Das deutsche New York ist: {}'.format(embeddings.most_similar(positive=['NewYork', 'Germany', 'city'], negative=['USA'])[0]))

# Wer ist eigentlich der Mozart der Physik?
print('Der Mozart der Physik ist: {}'.format(embeddings.most_similar(positive=['Mozart', 'physicist', 'person'], negative=['musician'])[0]))

# Welches Wort verhält sich zu 'singing' wie 'burnt' zu 'burning'?
print('burning:burnt wie singing:{}'.format(embeddings.most_similar(positive=['burning', 'burnt', 'singing'], negative=['burning'])[0]))

# Sind sich Deutschland und Frankreich ähnlicher oder Deutschland und Kanada?
print('Ähnlichkeit DE, FR: {}'.format(embeddings.distance('DE', 'FR')))
print('Ähnlichkeit DE, CAN: {}'.format(embeddings.distance('DE', 'CAN')))


Das deutsche New York ist: ('Berlin', 0.5106605291366577)
Der Mozart der Physik ist: ('Einstein', 0.5265765190124512)
burning:burnt wie singing:('sang', 0.6142075061798096)
Ähnlichkeit DE, FR: 0.5436527729034424
Ähnlichkeit DE, CAN: 0.7556446194648743


Bei der Interpretation der Ergebnisse ist jedoch Vorsicht geboten: Es werden zwar semantische Beziehungen abgebildet, aber die entsprechen möglicherweise nicht immer den Erwartungen.

Wie ähnlich sind sich zum Beispiel "Leben" und "Tod", "kalt" und "warm", "Norden" und "Süden"?

Sind die Ergebnisse wie erwartet? Warum (nicht)?

In [39]:
# TODO
print('Ähnlichkeit Leben, Tod: {}'.format(embeddings.similarity('life', 'death')))
print('Ähnlichkeit kalt, warm: {}'.format(embeddings.similarity('cold', 'warm')))
print('Ähnlichkeit Norden, Süden: {}'.format(embeddings.similarity('north', 'south')))

Ähnlichkeit Leben, Tod: 0.3618776500225067
Ähnlichkeit kalt, warm: 0.5953035354614258
Ähnlichkeit Norden, Süden: 0.9674535393714905


Die Embeddings sind nicht neutral, sondern spiegeln die Beziehungen wieder, die sich in den Trainingsdaten finden lassen.

In [44]:
# Wird Wissenschaft von Frauen oder Männern gemacht?
print('Wissenschaft wird gemacht von: {}'.format(embeddings.most_similar_to_given('science', ['man', 'woman'])))
# Sind Mörder eher Schwarze, Weiße oder Asiaten?
print('Mörder sind: {}'.format(embeddings.most_similar_to_given('murderers', ['black', 'white', 'asian'])))
# Was bleibt vom Mann, wenn die Intelligenz abgezogen wird?
print('Mann ohne Intelligenz: {}'.format(embeddings.most_similar(positive=['man'], negative=['intelligence'])))


Wissenschaft wird gemacht von: woman
Mörder sind: black
Mann ohne Intelligenz: [('woman', 0.525009274482727), ('boy', 0.4727959632873535), ('teenager', 0.4585689902305603), ('teenage_girl', 0.42294150590896606), ('girl', 0.3896019458770752), ('teen_ager', 0.3817293643951416), ('robber', 0.37824171781539917), ('Man', 0.3740370571613312), ('Robbery_suspect', 0.3724946975708008), ('suspected_purse_snatcher', 0.36925598978996277)]


## Aufgabe 2: Von Wörtern zu Sätzen
Nachdem wir uns mit der semantischen Repräsentation von Wörtern beschäftigt haben, wollen wir jetzt zu Sätzen übergehen. Wie bereits gesagt, wollen wir die Ähnlichkeit von Satzpaaren bestimmen. Dazu brauchen wir einen Testdatensatz.

### Aufgabe 2.1: Testdaten einlesen
Die Datei ```sts-train.csv``` enthält 5749 Satzpaare, deren Ähnlichkeit jeweils mit einem Score zwischen 0 (völlig unähnlich) und 5 (völlig ähnlich) bewertet wurde. Lest diese Datei in einen Pandas-Dataframe ein.

Zu beachten sind die folgenden Punkte:
* Bei den CSV-Dateien handelt es sich eigentlich strenggenommen um TSV-Dateien, d.h. die einzelnen Spalten sind durch Tab getrennt.
* Einige Zeilen enthalten unvollständige Quotes, die zu Fehlern beim Einlesen führen können. Diese sollen ignoriert werden. (```quoting=csv.QUOTE_NONE```)
* Zitat aus der zum Datenset gehörigen README: 
>Each file is encoded in utf-8 (a superset of ASCII), and has the following tab separated fields:  
>__genre filename year score sentence1 sentence2__  
>optionally there might be some license-related fields after sentence2.

Neben den uns interessierenden Spalten "score", "sentence1" und "sentence2" sind also noch weitere Spalten enthalten. Wir können den Parameter ```usecols``` verwenden, um die Indizes der uns interessierenden Spalten anzugeben (Zählung beginnt bei 1). Für die Spalten vergeben wir die Namen "score", "sentence1" und "sentence2".

In [50]:
import pandas as pd
import csv

testdata =  pd.read_csv('data/sts-train.csv', quoting=csv.QUOTE_NONE, sep='\t', encoding='utf-8', usecols=[0,1,2,3,4,5])

In [51]:
testdata.head()

,main-captions,MSRvid,2012test,0001,5.000,A plane is taking off.
0,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.
1,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.
2,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.
3,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.
4,main-captions,MSRvid,2012test,11,4.25,Some men are fighting.


### Aufgabe 2.2: Semantische Repräsentation von Sätzen
Genau wie für Wörter wollen wir auch die Semantik von Sätzen in einem Vektor der Dimension 300 abbilden. Dazu wählen wir einen recht einfachen Ansatz und mitteln die Vektoren der Wörter, aus denen der Satz besteht.

Schreibt eine Funktion, die einen Satz entgegennimmt und einen Vektor mit den gemittelten semantischen Embeddings zurückgibt. Wird ein leerer String übergeben oder enthält der übergebene Satz nur Wörter, für die keine Embeddings vorhanden sind, soll ein Nullvektor der Länge 300 zurückgegeben werden.

In [ ]:
import numpy as np
    
def sentence_to_vec(sentence):
    # TODO

In [ ]:
def sentence_to_vec_tests():
    if (np.zeros(300) != sentence_to_vec('')).any():
        return "Bei leerer Eingabe soll ein Nullvektor zurückgegeben werden"
    if (np.zeros(300) != sentence_to_vec('thereisnosuchword')).any():
        return "Wenn keine Embeddings gefunden werden, soll ein Nullvektor zurückgegeben werden."
    if (embeddings['word'] != sentence_to_vec('word')).any():
        return "Da stimmt was nicht."
    if ((embeddings['cat'] + embeddings['dog']) / 2 != sentence_to_vec('cat dog')).any():
        return "Die Funktion soll den Durchschnittswert der Vektoren berechnen."
    if (embeddings['word'] != sentence_to_vec('thereisnosuchword word')).any():
        return "Wörter, deren Embedding nicht bekannt ist, sollen nicht berücksichtigt werden."
    return "Grundlegende Tests bestanden."

In [ ]:
print(sentence_to_vec_tests())

### Aufgabe 2.3: Ähnlichkeitsberechnung
Um die Ähnlichkeit zweier Vektoren zu bestimmen, wird häufig die [Kosinusähnlichkeit](https://de.wikipedia.org/wiki/Kosinus-%C3%84hnlichkeit) verwendet. Diese kann Werte zwischen −1 (Vektoren sind genau entgegengerichtet) und 1 (Vektoren sind genau gleichgerichtet) annehmen. Weil wir die von uns berechneten Ähnlichkeitswerte aber mit denen der Testdaten vergleichen können wollen, müssen wir den Wertebereich so anpassen, dass wir Werte zwischen 0 und 5 erhalten. Dabei treffen wir die vereinfachende Annahme, dass eine Kosinusähnlichkeit kleiner-gleich 0 als unähnlich (Wert 0) zu werten ist.

Schreibt eine Funktion, die zwei Sätze entgegennimmt und Ähnlichkeitswerte im Bereich ```[0, 5]``` zurückgibt, basierend auf der Kosinusähnlichkeit der semantischen Vektoren der Sätze.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def compute_similarity_score(sentence_1, sentence_2):
    # TODO

In [ ]:
import math
def compute_similarity_score_test():
    if not (math.isclose(compute_similarity_score("hot dog", "hot dog"), 5.0)):
        return "Identische Sätze sollten einen Ähnlichkeitswert von 5 haben."
    if not (math.isclose(compute_similarity_score("dog hot", "hot dog"), 5.0)):
        return "Sätze, die sich nur in der Anordnung der Wörter unterscheiden, sollten einen Ähnlichkeitswert von 5 haben."
    if not (math.isclose(compute_similarity_score("nosuchword", "word"), 0.0)):
        return "Der Vergleich eines unbekannten Wortes mit einem bekannten, sollte zu einem Ähnlichkeitswert von 0 führen."
    if (math.isclose(compute_similarity_score("bread", "cake"), 0.0)):
        return "Verwandte Worte sollten einen Ähnlichkeitswert größer Null aufweisen."
    return "Grundlegende Tests bestanden."

compute_similarity_score_test()

## Aufgabe 3: Validierung
Im letzten Schritt wollen wir bestimmen, wie gut unsere Ähnlichkeitsberechnung funktioniert, d.h. wie stark die von uns berechneten Ähnlichkeitswerte für die Satzpaare aus dem Testdatensatz mit den erwarteten Werten übereinstimmen.

Dazu verwenden wir den [Korrelationskoeffizienten](https://de.wikipedia.org/wiki/Korrelationskoeffizient), der uns in ```scipy``` als [```pearsonr```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html) zur Verfügung steht.

Schreibt eine Funktion, die zwei Listen mit Ähnlichkeitswerten entgegennimmt und für diese den Pearsonscore berechnet. Wendet die Funktion auf die für die Testdaten erwarteten und die von uns berechneten Ähnlichkeitsscores an und interpretiert das Ergebnis.

In [ ]:
import numpy as np
from scipy import stats
def calculate_pearson_score(expected, actual):
    # TODO

In [ ]:
#TODO: Anwenden


In [6]:
!pip list

Package            Version 
------------------ --------
alembic            1.0.8   
asn1crypto         0.24.0  
async-generator    1.10    
attrs              19.1.0  
backcall           0.1.0   
beautifulsoup4     4.7.1   
bleach             3.1.0   
bokeh              1.0.4   
boto               2.49.0  
boto3              1.9.145 
botocore           1.12.145
certifi            2019.3.9
cffi               1.12.3  
chardet            3.0.4   
Click              7.0     
cloudpickle        0.8.1   
conda              4.6.14  
cryptography       2.6.1   
cycler             0.10.0  
Cython             0.29.7  
cytoolz            0.9.0.1 
dask               1.1.5   
decorator          4.4.0   
defusedxml         0.5.0   
dill               0.2.9   
distributed        1.28.0  
docutils           0.14    
entrypoints        0.3     
fastcache          1.1.0   
gensim             3.7.3   
gmpy2              2.0.8   
h5py               2.9.0   
heapdict           1.0.0   
idna               2